# Subnet Design Tutorial
<i>Adapted for use with FABRIC from [Designing Subnets](https://witestlab.poly.edu/blog/designing-subnets/)
    
In this tutorial you will learn:
* How to assign Subnets in a slice
    
<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)

Subnets are divisions of computer networks that reduce the size of the broadcast domain or create boundaries for organization, administration and security. Routing uses an IP address and a subnet number, this subnet number tells whether packets exist within the network currently being traversed. Manual configuration is usually used when creating these addresses, making these addresses known to the routers. When designing subnets you will have to consider problems such as limited IP space, accomodating all hosts and how traffic will be routed between subnets. 
    
The subnet mask is the first x bits of an address showing which IP addresses may be found in a network. Each IP address on the network will share the same first X bits, and vary in whatever is left. For example in IPV4 an address can carry 32 bits, so an address such as 10.189.0.0/26 will use the first 16 bits as the subnet and all addresses on the network will look like 10.189.x.x .  The amount of hosts that a subnet can carry is 2^y where y is the remaining bits on an address. 

## 1. Design the Experiment
### 1.1 Reserve Resources

#### Import the Fabric API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

####  Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.


In [ ]:
slice_name = 'SubnetExample'
try:
    slice = fablib.get_slice(slice_name)
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")

## 2. Implement design on each LAN
Here the network interface on each host and router on each of the three LANS will be configured.

Within each LAN an IP address and subnet mask will be assigned to every interface connected to the LAN.

In [ ]:
slice = fablib.get_slice("SubnetExample")
slice.list_nodes()

#### IP ranges for our subnet masks

Here we define our ranges for each of our subnets.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
subnetMid = IPv4Network("10.10.100.0/24")
available_ips = list(subnetMid)[1:]
total_hosts = int(str(available_ips[-1]).split('.')[-1]) - int(str(available_ips[0]).split('.')[-1])
print("Router Lan, supports: " + str(total_hosts) + " hosts")
print(str(available_ips[0]) + " - " + str(available_ips[-1])+"\n")

subnetA = IPv4Network("10.189.24.0/26")
available_ips = list(subnetA)[1:]
total_hosts = int(str(available_ips[-1]).split('.')[-1]) - int(str(available_ips[0]).split('.')[-1])
print("Lan A, supports: " + str(total_hosts) + " hosts")
print(str(available_ips[0]) + " - " + str(available_ips[-1])+"\n")

subnetB = IPv4Network("10.189.24.128/25")
available_ips = list(subnetB)[1:]
total_hosts = int(str(available_ips[-1]).split('.')[-1]) - int(str(available_ips[0]).split('.')[-1])
print("Lan B, supports: " + str(total_hosts) + " hosts")
print(str(available_ips[0]) + " - " + str(available_ips[-1])+"\n")

subnetC = IPv4Network("10.189.24.64/27")
available_ips = list(subnetC)[1:]
total_hosts = int(str(available_ips[-1]).split('.')[-1]) - int(str(available_ips[0]).split('.')[-1])
print("Lan C, supports: " + str(total_hosts) + " hosts")
print(str(available_ips[0]) + " - " + str(available_ips[-1])+"\n")

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    #Lan A
    LanA = slice.get_node(name="LanA") 
    Romeo = slice.get_node(name="Romeo")
    Juliet = slice.get_node(name="Juliet")
    
    #Lan B
    LanB = slice.get_node(name="LanB") 
    Othello = slice.get_node(name="Othello")
    Desdemona = slice.get_node(name="Desdemona")
    
    #Lan C
    LanC = slice.get_node(name="LanC")
    Hamlet = slice.get_node(name="Hamlet")
    Ophelia = slice.get_node(name="Ophelia")    
    
    #Router Lan setup
    LanA_iface = LanA.get_interface(network_name="Inter")
    LanA_iface.ip_addr_add(addr="10.10.100.1", subnet=subnetMid)
    
    LanB_iface = LanB.get_interface(network_name="Inter")
    LanB_iface.ip_addr_add(addr="10.10.100.2", subnet=subnetMid)
    
    LanC_iface = LanC.get_interface(network_name="Inter")
    LanC_iface.ip_addr_add(addr="10.10.100.3", subnet=subnetMid)
    
    #Lan A setup
    LanA_iface = LanA.get_interface(network_name="Lan_A")
    LanA_iface.ip_addr_add(addr="10.189.24.10", subnet=subnetA)
    
    Romeo_iface = Romeo.get_interface(network_name="Lan_A")
    Romeo_iface.ip_addr_add(addr="10.189.24.20", subnet=subnetA)
    
    Juliet_iface = Juliet.get_interface(network_name="Lan_A")
    Juliet_iface.ip_addr_add(addr="10.189.24.30", subnet=subnetA)
    
    #Lan B setup
    LanB_iface = LanB.get_interface(network_name="Lan_B")
    LanB_iface.ip_addr_add(addr="10.189.24.130", subnet=subnetB)
    
    Othello_iface = Othello.get_interface(network_name="Lan_B")
    Othello_iface.ip_addr_add(addr="10.189.24.140", subnet=subnetB)
    
    Desdemona_iface = Desdemona.get_interface(network_name="Lan_B")
    Desdemona_iface.ip_addr_add(addr="10.189.24.150", subnet=subnetB)
    
    #Lan C setup
    LanC_iface = LanC.get_interface(network_name="Lan_C")
    LanC_iface.ip_addr_add(addr="10.189.24.70", subnet=subnetC)
    
    Hamlet_iface = Hamlet.get_interface(network_name="Lan_C")
    Hamlet_iface.ip_addr_add(addr="10.189.24.80", subnet=subnetC)
    
    Ophelia_iface = Ophelia.get_interface(network_name="Lan_C")
    Ophelia_iface.ip_addr_add(addr="10.189.24.90", subnet=subnetC)
    
except Exception as e:
    print(f"Exception: {e}")

#### Enable forwarding on all nodes

In [ ]:
for node in slice.get_nodes():
    node.execute("sudo apt-get update -y; sudo apt-get install traceroute -y; sudo sysctl -w net.ipv4.ip_forward=1")

#### Test connection

We will ping Ophelia 5 times to test if the connection is available.

In [ ]:
print(Romeo.execute("ping -c 5 10.189.24.90")[0])

#### Add routing to all nodes
By default the routing tables on each node can't communicate with each other, this block of code adds all other subnets to their routing table.

In [ ]:
#Lan A
LanA.execute("sudo ip route add 10.189.24.64/27 via 10.10.100.3;sudo ip route add 10.189.24.128/25 via 10.10.100.2;")
Romeo.execute("sudo ip route add 10.189.24.64/27 via 10.189.24.10;sudo ip route add 10.189.24.128/25 via 10.189.24.10;")
Juliet.execute("sudo ip route add 10.189.24.64/27 via 10.189.24.10;sudo ip route add 10.189.24.128/25 via 10.189.24.10;")

#Lan B
LanB.execute("sudo ip route add 10.189.24.0/26 via 10.10.100.1; sudo ip route add 10.189.24.64/27 via 10.10.100.3;")
Othello.execute("sudo ip route add 10.189.24.0/26 via 10.189.24.130; sudo ip route add 10.189.24.64/27 via 10.189.24.130;")
Desdemona.execute("sudo ip route add 10.189.24.0/26 via 10.189.24.130; sudo ip route add 10.189.24.64/27 via 10.189.24.130;")

#Lan C
LanC.execute("sudo ip route add 10.189.24.0/26 via 10.10.100.1; sudo ip route add 10.189.24.128/25 via 10.10.100.2;")
Hamlet.execute("sudo ip route add 10.189.24.0/26 via 10.189.24.70;sudo ip route add 10.189.24.128/25 via 10.189.24.70;")
Ophelia.execute("sudo ip route add 10.189.24.0/26 via 10.189.24.70;sudo ip route add 10.189.24.128/25 via 10.189.24.70;")

#### Test connection

We will ping Ophelia 5 times to make sure the connection is available.

In [ ]:
print(Romeo.execute("ping -c 5 10.189.24.90")[0])

#### Print the route from Romeo to Ophelia

`Traceroute` will show the hops used to get from the first node (Romeo) to the destination (Ophelia).

In [ ]:
print(Romeo.execute("traceroute 10.189.24.90")[0])

## 3. Cleanup Resources
### 3.1 Delete Slice

In [ ]:
try:
    slice = fablib.get_slice("SubnetExample")
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")